In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Function to parse the generated text and extract the triplets
def extract_triplets(text):
    triplets = []
    for i in text.replace("<s>", "").replace("</s>", "").split("$"):
      dd = i.split("|")
      res_dict = {}
      res_dict["head"] = dd[0].replace("[", "").replace("]", "")
      res_dict["relation"] = dd[1].replace("[", "").replace("]", "")
      res_dict["tail"] = dd[2].replace("[", "").replace("]", "")
      triplets.append(res_dict)
    return triplets

## Example 1

In [3]:
from transformers.pipelines import text2text_generation
from transformers import pipeline

triplet_extractor = pipeline('text2text-generation', model='ibm/knowgl-large', tokenizer='ibm/knowgl-large')

# We need to use the tokenizer manually since we need special tokens.
str_data = """Stocks continued to recover from oversold levels, with traders weighing renewed speculation that global central banks could moderate their hawkish stance to prevent a hard landing. The S&P 500 is having its best two-day surge since November 2020. On top of the obvious short squeeze, a soft economic reading gave bulls a reason for hope when it comes to Federal Reserve policy. US job openings sank to a 14-month low—which may fit well with a central bank that’s very concerned about a hot jobs market. Bond yields fell with the dollar. Traders also kept a close eye on Tesla Inc., which pared gains substantially on news that Elon Musk is proposing to buy Twitter Inc. for the original offer price of $54.20 a share. At midday, the S&P 500 was up 2.37% and the Dow was up 2.15%. The debate over peak hawkishness has intensified after a dovish surprise from Australia’s central bank and bond buying by the Bank of England. The idea of a Fed pivot, however, has been met with a lot of skepticism. For one, there’s the perception that not much has fundamentally changed to sway officials from their primary goal to knock down inflation. Then, there’s the fact that stock pessimism reached such extreme levels that a bounce would be just a matter of when. For markets that had been “nearly one-sided,” the liquidation of those positions is a big reason to squeeze in the other direction so vigorously. For one, there’s the perception that not much has fundamentally changed to sway officials from their primary goal to knock down inflation. Then, there’s the fact that stock pessimism reached such extreme levels that a bounce would be just a matter of when. For markets that had been “nearly one-sided,” the liquidation of those positions is a big reason to squeeze in the other direction so vigorously."""

import nltk
nltk.download('punkt')
data = nltk.tokenize.sent_tokenize(str_data)
data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['Stocks continued to recover from oversold levels, with traders weighing renewed speculation that global central banks could moderate their hawkish stance to prevent a hard landing.',
 'The S&P 500 is having its best two-day surge since November 2020.',
 'On top of the obvious short squeeze, a soft economic reading gave bulls a reason for hope when it comes to Federal Reserve policy.',
 'US job openings sank to a 14-month low—which may fit well with a central bank that’s very concerned about a hot jobs market.',
 'Bond yields fell with the dollar.',
 'Traders also kept a close eye on Tesla Inc., which pared gains substantially on news that Elon Musk is proposing to buy Twitter Inc. for the original offer price of $54.20 a share.',
 'At midday, the S&P 500 was up 2.37% and the Dow was up 2.15%.',
 'The debate over peak hawkishness has intensified after a dovish surprise from Australia’s central bank and bond buying by the Bank of England.',
 'The idea of a Fed pivot, however, has been 

In [4]:
for i in data:
  extracted_text = triplet_extractor.tokenizer.batch_decode([triplet_extractor(i, return_tensors=True, return_text=False)[0]["generated_token_ids"]])
  print(extracted_text[0])
  extracted_triplets = extract_triplets(extracted_text[0])
  print(extracted_triplets)

<s><s>[(hawkish#Hawkish monetary policy#monetary policy)|facet of|(central bank#Central bank#bank)]</s>
[{'head': '(hawkish#Hawkish monetary policy#monetary policy)', 'relation': 'facet of', 'tail': '(central bank#Central bank#bank)'}]
<s><s>[(November 2020#November 2020 S&P 500 Index rally#stock market rally)|part of|(S&P 500#S&P 500 Index#stock market index)]</s>
[{'head': '(November 2020#November 2020 S&P 500 Index rally#stock market rally)', 'relation': 'part of', 'tail': '(S&P 500#S&P 500 Index#stock market index)'}]
<s><s>[(Federal Reserve#Federal Reserve#central bank)|instance of|(policy#Central bank#bank)]</s>
[{'head': '(Federal Reserve#Federal Reserve#central bank)', 'relation': 'instance of', 'tail': '(policy#Central bank#bank)'}]
<s><s>[(US job openings#Job openings in the United States#None)|facet of|(hot jobs market#Great Recession#recession)]</s>
[{'head': '(US job openings#Job openings in the United States#None)', 'relation': 'facet of', 'tail': '(hot jobs market#Great 

## Example 2

In [5]:
str_data = """Bonds are a basic form of investment that typically include a straightforward financial agreement between issuer and purchaser. Nevertheless, the terminology surrounding bonds is unique and rather extensive. Much of the specialized vocabulary surrounding bonds is designed to convey the concept that a bond is similar to other financial instruments in that it is an investment that can be bought and sold. Much of this unique terminology will be covered later in this chapter, but we can set out some of the basics here with an example."""
data = nltk.tokenize.sent_tokenize(str_data)
data

['Bonds are a basic form of investment that typically include a straightforward financial agreement between issuer and purchaser.',
 'Nevertheless, the terminology surrounding bonds is unique and rather extensive.',
 'Much of the specialized vocabulary surrounding bonds is designed to convey the concept that a bond is similar to other financial instruments in that it is an investment that can be bought and sold.',
 'Much of this unique terminology will be covered later in this chapter, but we can set out some of the basics here with an example.']

In [6]:
for i in data:
  extracted_text = triplet_extractor.tokenizer.batch_decode([triplet_extractor(i, return_tensors=True, return_text=False)[0]["generated_token_ids"]])
  print(extracted_text[0])
  extracted_triplets = extract_triplets(extracted_text[0])
  print(extracted_triplets)

<s><s>[(Bond#Bond (finance)#loan)|subclass of|(investment#Investment#purchasing)]</s>
[{'head': '(Bond#Bond (finance)#loan)', 'relation': 'subclass of', 'tail': '(investment#Investment#purchasing)'}]
<s><s>[(bond#Bond (mathematics)#mathematical object)|said to be the same as|(unique#Bond (mathematics)#mathematical object)]$[(unique#Bond (mathematics)#mathematical object)|said to be the same as|(bond#Bond (mathematics)#mathematical object)]</s>
[{'head': '(bond#Bond (mathematics)#mathematical object)', 'relation': 'said to be the same as', 'tail': '(unique#Bond (mathematics)#mathematical object)'}, {'head': '(unique#Bond (mathematics)#mathematical object)', 'relation': 'said to be the same as', 'tail': '(bond#Bond (mathematics)#mathematical object)'}]
<s><s>[(investment#Investment#purchasing)|subclass of|(financial instrument#Financial instrument#contract)]</s>
[{'head': '(investment#Investment#purchasing)', 'relation': 'subclass of', 'tail': '(financial instrument#Financial instrument#